In [10]:
import pandas as pd
import numpy as np
from catboost import Pool, CatBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import TimeSeriesSplit 
import optuna 
import json
from sklearn.metrics import mean_absolute_error

In [11]:
#Load inn datasets
X_test  = pd.read_parquet('../data/prepared_datasets/only_y_cleaned/X_test.parquet')
X_train = pd.read_parquet('../data/prepared_datasets/only_y_cleaned/X_train.parquet')
y_train = pd.read_parquet('../data/prepared_datasets/only_y_cleaned/Y_train.parquet')
y_train_a = pd.read_parquet('../data/prepared_datasets/only_y_cleaned/Y_train_a.parquet')
y_train_b = pd.read_parquet('../data/prepared_datasets/only_y_cleaned/Y_train_b.parquet')
y_train_c = pd.read_parquet('../data/prepared_datasets/only_y_cleaned/Y_train_c.parquet')

In [12]:
def splitting_def(df):
    date_range_1 = (df.index >= '2020-05-01') & (df.index <= '2020-06-25')
    date_range_2 = (df.index >= '2023-05-01') & (df.index <= '2023-06-15')

    # Combine the date ranges to create the test set
    test_set = df[date_range_1 | date_range_2]

    # The rest of the data will be your training set
    training_set = df[~(date_range_1 | date_range_2)]
    
    # Splitting the test_set into X_test and y_test
    X_test = test_set.drop("pv_measurement", axis=1)
    y_test = test_set['pv_measurement']  # Assuming 'pv_measurement' is your target variable

    # Splitting the training_set into X_train and y_train
    X_train = training_set.drop("pv_measurement", axis=1)
    y_train = training_set['pv_measurement']
    
    return X_train, X_test, y_train, y_test

X_train_new_a, X_test_new_a, y_train_new_a, y_test_a = splitting_def(pd.concat([X_train[X_train["location"] == "A"].drop("location", axis=1), y_train_a], axis=1))
X_train_new_b, X_test_new_b, y_train_new_b, y_test_b = splitting_def(pd.concat([X_train[X_train["location"] == "B"].drop("location", axis=1), y_train_b], axis=1))
X_train_new_c, X_test_new_c, y_train_new_c, y_test_c = splitting_def(pd.concat([X_train[X_train["location"] == "C"].drop("location", axis=1), y_train_c], axis=1))

In [13]:
#Create a pool of data
train_pool_a = Pool(X_train_new_a, y_train_new_a)
train_pool_b = Pool(X_train_new_b, y_train_new_b)
train_pool_c = Pool(X_train_new_c, y_train_new_c)


test_pool_a = Pool(X_test_new_a) 
test_pool_b = Pool(X_test_new_b) 
test_pool_c = Pool(X_test_new_c) 

In [14]:
#For location A
def objective(trial, X_train, y_train):
    params = {
        "iterations": trial.suggest_int("iterations", 300, 3000),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 13),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
        "l2_leaf_reg": trial.suggest_int("l2_leaf_reg", 2, 10),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.3, 1.0),
        "random_strength": trial.suggest_float("random_strength", 0.3, 1.0),
        "border_count": trial.suggest_int("border_count", 1, 1000),
        "rsm": trial.suggest_float("rsm", 0.05, 1),
    }

    catboost_model_a = CatBoostRegressor(**params, verbose=100)
    catboost_model_a.fit(train_pool_a)
    pred_a = pd.DataFrame(catboost_model_a.predict(test_pool_a))
    MAE_a = mean_absolute_error(y_test_a, pred_a)
    return MAE_a

study = optuna.create_study(direction='minimize')
study.optimize(lambda trial: objective(trial, X_train, y_train), n_trials=80)

[I 2023-11-07 08:23:56,934] A new study created in memory with name: no-name-73d88897-e286-457b-a757-7b14b380bcbe


0:	learn: 1127.5731913	total: 83.1ms	remaining: 3m 5s
100:	learn: 538.7490122	total: 7.63s	remaining: 2m 41s
200:	learn: 441.8060017	total: 14.5s	remaining: 2m 26s
300:	learn: 418.8111607	total: 21.1s	remaining: 2m 15s
400:	learn: 405.2827112	total: 27.4s	remaining: 2m 5s
500:	learn: 394.9905655	total: 34.3s	remaining: 1m 58s
600:	learn: 385.7341918	total: 41.2s	remaining: 1m 51s
700:	learn: 377.0242199	total: 47.9s	remaining: 1m 44s
800:	learn: 369.4404858	total: 54.8s	remaining: 1m 37s
900:	learn: 362.4334843	total: 1m 1s	remaining: 1m 31s
1000:	learn: 355.8196707	total: 1m 9s	remaining: 1m 25s
1100:	learn: 349.3995260	total: 1m 15s	remaining: 1m 17s
1200:	learn: 343.1558157	total: 1m 22s	remaining: 1m 10s
1300:	learn: 336.9680248	total: 1m 28s	remaining: 1m 3s
1400:	learn: 331.4245725	total: 1m 35s	remaining: 56.4s
1500:	learn: 325.8223155	total: 1m 41s	remaining: 49.4s
1600:	learn: 320.8133714	total: 1m 47s	remaining: 42.5s
1700:	learn: 315.6822596	total: 1m 54s	remaining: 35.7s
18

[I 2023-11-07 08:26:23,095] Trial 0 finished with value: 341.6693006096711 and parameters: {'iterations': 2233, 'learning_rate': 0.013224540310593091, 'depth': 8, 'min_data_in_leaf': 87, 'l2_leaf_reg': 2, 'bagging_temperature': 0.9847191385516187, 'random_strength': 0.3393670193500201, 'border_count': 541, 'rsm': 0.7329034008433479}. Best is trial 0 with value: 341.6693006096711.


2232:	learn: 291.6005652	total: 2m 25s	remaining: 0us
0:	learn: 1131.9739257	total: 37.4ms	remaining: 26.3s
100:	learn: 679.4064279	total: 2.36s	remaining: 14s
200:	learn: 518.4146947	total: 5.17s	remaining: 12.9s
300:	learn: 468.6249305	total: 8.24s	remaining: 11s
400:	learn: 447.6762159	total: 10.8s	remaining: 8.17s
500:	learn: 436.0794362	total: 13.9s	remaining: 5.62s
600:	learn: 428.7656842	total: 16.9s	remaining: 2.88s


[I 2023-11-07 08:26:43,286] Trial 1 finished with value: 356.67992569042394 and parameters: {'iterations': 703, 'learning_rate': 0.009286387680388768, 'depth': 8, 'min_data_in_leaf': 59, 'l2_leaf_reg': 4, 'bagging_temperature': 0.885009419955769, 'random_strength': 0.9236507833980463, 'border_count': 739, 'rsm': 0.08080155019915247}. Best is trial 0 with value: 341.6693006096711.


...

0:	learn: 1133.3681150	total: 236ms	remaining: 10m 27s
100:	learn: 706.2452410	total: 22.8s	remaining: 9m 38s
200:	learn: 521.6435447	total: 47.5s	remaining: 9m 40s
300:	learn: 442.0707238	total: 1m 10s	remaining: 9m 12s
400:	learn: 402.7629601	total: 1m 33s	remaining: 8m 48s
500:	learn: 379.7082244	total: 1m 57s	remaining: 8m 25s
600:	learn: 364.4554563	total: 2m 20s	remaining: 8m
700:	learn: 351.1812152	total: 2m 43s	remaining: 7m 36s
800:	learn: 338.7427154	total: 3m 6s	remaining: 7m 12s
900:	learn: 327.1966008	total: 3m 29s	remaining: 6m 49s
1000:	learn: 316.6172405	total: 3m 53s	remaining: 6m 27s
1100:	learn: 306.8749880	total: 4m 16s	remaining: 6m 3s
1200:	learn: 297.8539227	total: 4m 40s	remaining: 5m 40s
1300:	learn: 289.6201029	total: 5m 3s	remaining: 5m 17s
1400:	learn: 281.2540705	total: 5m 27s	remaining: 4m 54s
1500:	learn: 273.6015816	total: 5m 50s	remaining: 4m 31s
1600:	learn: 266.2127999	total: 6m 13s	remaining: 4m 7s
1700:	learn: 259.2922023	total: 6m 37s	remaining: 3m

[I 2023-11-07 11:54:59,975] Trial 52 finished with value: 331.6629828139493 and parameters: {'iterations': 2661, 'learning_rate': 0.006968859176463766, 'depth': 13, 'min_data_in_leaf': 94, 'l2_leaf_reg': 3, 'bagging_temperature': 0.5383095167977268, 'random_strength': 0.8247738689823597, 'border_count': 833, 'rsm': 0.32772800751053965}. Best is trial 51 with value: 328.3860772466676.


0:	learn: 1133.2985585	total: 294ms	remaining: 7m 27s
100:	learn: 698.6318200	total: 26.7s	remaining: 6m 15s
200:	learn: 513.5667495	total: 52.7s	remaining: 5m 46s
300:	learn: 434.1472556	total: 1m 19s	remaining: 5m 20s
400:	learn: 395.5552319	total: 1m 45s	remaining: 4m 54s
500:	learn: 371.7547875	total: 2m 11s	remaining: 4m 28s
600:	learn: 353.9862200	total: 2m 38s	remaining: 4m 2s
700:	learn: 339.9323735	total: 3m 4s	remaining: 3m 36s
800:	learn: 326.6252840	total: 3m 31s	remaining: 3m 10s
900:	learn: 315.0840523	total: 3m 57s	remaining: 2m 43s
1000:	learn: 304.0848548	total: 4m 23s	remaining: 2m 17s


[W 2023-11-07 11:59:31,029] Trial 53 failed with parameters: {'iterations': 1522, 'learning_rate': 0.007060797972679036, 'depth': 13, 'min_data_in_leaf': 95, 'l2_leaf_reg': 2, 'bagging_temperature': 0.5353354278609662, 'random_strength': 0.8290864611090117, 'border_count': 842, 'rsm': 0.388184021376477} because of the following error: KeyboardInterrupt('').
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/var/tmp/ipykernel_2241/2326973077.py", line 22, in <lambda>
    study.optimize(lambda trial: objective(trial, X_train, y_train), n_trials=80)
  File "/var/tmp/ipykernel_2241/2326973077.py", line 16, in objective
    catboost_model_a.fit(train_pool_a)
  File "/opt/conda/lib/python3.10/site-packages/catboost/core.py", line 5703, in fit
    return self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, basel

KeyboardInterrupt: 

In [10]:
#to output the best paramaters
print(study.best_params)

#to output the best score returned from the trials
print(study.best_value)


with open("optuna-best-parameters_a.txt", "w") as file:
    file.write("Best paramaters: \n")
    file.write(json.dumps(study.best_params))  # Write the first string followed by a newline character
    file.write("\n")
    file.write("best score MAE: \n")
    file.write(json.dumps(study.best_value))  # Write the second string followed by a newline character

{'iterations': 1852, 'learning_rate': 0.0018854412552105353, 'depth': 9, 'colsample_bylevel': 0.7366562974962199, 'min_data_in_leaf': 4, 'l2_leaf_reg': 3, 'has-time': True, 'bagging_temperature': 0.9525931510378696, 'random_strength': 0.30613129187014626, 'border_count': 993, 'rsm': 0.21119141087462187, 'nan_mode': 'min'}
351.34628571238125
